# Построение модели для выбора подходящего тарифа
______

Задача - добиться показателя Accuracy на тестовой выборке выше 0.75.

#### Данное исследование разделим на несколько частей.
______

### Часть 1. Изучение общей информации:
* [1. Изучение файла с данными, получение общей информации, загрузка библиотек.](#1-bullet)


### Часть 2. Подготовка данных:
* [1. Разделение исходных данных на обучающую, валидационную и тестовую выборки.](#2-bullet)


### Часть 3. Исследование качества разных моделей:
* [1. Качество решающего дерева.](#3-bullet)
* [2. Качество модели случайный лес.](#4-bullet)
* [3. Качество логистической регрессии.](#5-bullet)

### Часть 4. Проверка качества модели на тестовой выборке:
* [1. Проверка качества модели на тестовой выборке.](#6-bullet)

#### Описание данных:

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

#### 1-bullet 

### Часть 1. Изучение общей информации:

1. Изучение файла с данными, получение общей информации, загрузка библиотек.

In [1]:
# загружаем все необходимые для данного исследования библиотеки
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# открываем файл с данными и получаем общую информацию по нему
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод

- Загрузили необходимые библиотеки, открыли файл и получили общую информацию. 
- В таблице 5 колонок и 3214 строк. Пропусков не имеется. Тип данных во всех столбцах корректный. 
- Данные пригодны для исследования. Предобработка не требуется.

####  2-bullet

### Часть 2. Подготовка данных:

1. Разделение исходных данных на обучающую, валидационную и тестовую выборки.

In [2]:
# сохраним в отдельных переменных признаки и целевой признак
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

# сохраним значение параметра random_state в переменной 
my_random_state = 123

# разделим на две выборки: на тестовую и для обучения с валидацией
features_train_valid, features_test, target_train_valid, target_test = (
    train_test_split(features, target, test_size=0.20, 
                     random_state=my_random_state))

# разделим на две выборки обучающую с валидационной выборку
features_train, features_valid, target_train, target_valid = (
    train_test_split(features_train_valid, target_train_valid, 
                     test_size=0.25, random_state=my_random_state))

### Вывод

Сначала в отдельные переменные сохранили признаки и целевой признак. Целевой признак у нас - столбец 'is_ultra', поскольку цель исследования - предложить один из двух тарифов.

Чтобы разделить исходные данные на три выборки сначала разделим на две выборки: обучающую вместе с валидационной и тестовую. А затем уже обучающую вместе с валидационной разделим на обучающую и валидационную.

Чтобы соотношение выборок в итоге оказалось 3:1:1, при первом разделении test_size зададим 0.2, а при втором разделении 0.25. Так у нас в итоге получится нужное нам соотношение выборок.

#### 3-bullet

### Часть 3. Исследование качества разных моделей:

1. Качество решающего дерева.

- построим модель решающее дерево, используя разные гиперпараметры глубины, при этом гиперпараметр random_state зафиксируем на одном значении.

- получим предсказания и рассчитаем качество модели по валидационной выборке и тестовой выборкам с разными значениями максимальной глубины.

- найдем наилучшую по качеству модель и ее гиперпараметры.

In [3]:
# создадим функцию, которая будет перебирать гиперпараметр 
# максимальной глубины дерева, возьмем от 1 до 20

best_model = 0
best_result = 0
for depth in range(1, 31):
    # сохраним модель решающего дерева    
    model_tree = DecisionTreeClassifier(random_state=my_random_state, 
                                        max_depth=depth) 
    model_tree.fit(features_train, target_train) # обучим модель
    # получим предсказания модели по валидационной выборке
    predictions = model_tree.predict(features_valid) 
    result_1 = accuracy_score(target_valid, predictions) # получим качество модели
    # получим предсказания модели по обучающей выборке
    predictions = model_tree.predict(features_train) 
    result_2 = accuracy_score(target_train, predictions) # получим качество модели
    
    # выведем на экран качество моделей по валидационной и обучающей выборкам
    print('Accuracy на валид-ой выборке:', result_1, 
          "Максим-ая глубина:", depth)
    print('Accuracy на обуч-щей выборке:', result_2, 
          "Максим-ая глубина:", depth)
    
    # найдем наилучшую модель
    if result_1 > best_result:
        best_model = model_tree
        best_result = result_1
        best_depth = depth
        
# выведем на экран наилучшее качество модели и глубину
print('Accuracy наилучшей модели на валидационной выборке:', best_result, 
      "Максимальная глубина:", best_depth)

Accuracy на валид-ой выборке: 0.7356143079315708 Максим-ая глубина: 1
Accuracy на обуч-щей выборке: 0.7505186721991701 Максим-ая глубина: 1
Accuracy на валид-ой выборке: 0.7651632970451011 Максим-ая глубина: 2
Accuracy на обуч-щей выборке: 0.7816390041493776 Максим-ая глубина: 2
Accuracy на валид-ой выборке: 0.7838258164852255 Максим-ая глубина: 3
Accuracy на обуч-щей выборке: 0.7971991701244814 Максим-ая глубина: 3
Accuracy на валид-ой выборке: 0.7822706065318819 Максим-ая глубина: 4
Accuracy на обуч-щей выборке: 0.8003112033195021 Максим-ая глубина: 4
Accuracy на валид-ой выборке: 0.7791601866251944 Максим-ая глубина: 5
Accuracy на обуч-щей выборке: 0.8184647302904564 Максим-ая глубина: 5
Accuracy на валид-ой выборке: 0.7884914463452566 Максим-ая глубина: 6
Accuracy на обуч-щей выборке: 0.8309128630705395 Максим-ая глубина: 6
Accuracy на валид-ой выборке: 0.7838258164852255 Максим-ая глубина: 7
Accuracy на обуч-щей выборке: 0.8428423236514523 Максим-ая глубина: 7
Accuracy на валид-ой

### Вывод

- перебирая различные значения максимальной глубины дерева, получили значение качества на валидационной выборке Accuracy: 0.7947122861586314 при максимальной глубине: 9.

- также проверили предсказания модели по выборке, по которой модель обучалась. При максимальной глубине 26 Accuracy равно единице.

- как видим, на лицо переобучение модели, поскольку качество на валидационной выборке значительно ниже, чем на обучающей.

- при этом скорость работы модели нормальная.

#### 4-bullet

### Часть 3. Исследование качества разных моделей:

2. Качество модели случайный лес.

- построим модель случайный лес, используя разные гиперпараметры количества деревьев и глубины, при этом гиперпараметр random_state зафиксируем на одном значении.

- получим предсказания и рассчитаем качество модели по валидационной выборке и тестовой выборкам с разными гиперпараметрами.

- найдем наилучшую по качеству модель и ее гиперпараметры.

In [4]:
# создадим функцию, которая будет перебирать гиперпараметры 
# количества деревьев от 10 до 50 с шагом в 10 и 
# максимальной глубины дерева от 1 до 10

best_model = 0
best_result = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        # сохраним модель случайный лес с искомыми гиперпараметрами
        model_rf = RandomForestClassifier(random_state=my_random_state, 
                                         n_estimators=est, 
                                         max_depth=depth)
        model_rf.fit(features_train, target_train) # обучим модель
        # получим предсказания по валидационной выборке
        predictions_valid = model_rf.predict(features_valid)  
        result_1 = model_rf.score(features_valid, target_valid) # посчитаем качество
        # получим предсказания по обучающей выборке        
        predictions_train = model_rf.predict(features_train)  
        result_2 = model_rf.score(features_train, target_train) # посчитаем качество
        
        # выведем на экран качество по валидационной и обучающей выборкам
        print("Accuracy на валид-ой выборке:", result_1,
              "Кол-во деревьев:", est, "Максим-ая глубина:", depth)
        print("Accuracy на обуч-щей выборке:", result_2,
              "Кол-во деревьев:", est, "Максим-ая глубина:", depth)
        
        # найдем лучшую модель
        if result_1 > best_result:
            best_model = model_rf
            best_result = result_1
            best_est = est
            best_depth = depth

# выведем на экран качество лучшей модели, количество деревьев и глубина
print("Accuracy наилучшей модели на валидационной выборке:", best_result, 
      "Количество деревьев:", best_est, "Максимальная глубина:", best_depth)      

Accuracy на валид-ой выборке: 0.7433903576982893 Кол-во деревьев: 10 Максим-ая глубина: 1
Accuracy на обуч-щей выборке: 0.7515560165975104 Кол-во деревьев: 10 Максим-ая глубина: 1
Accuracy на валид-ой выборке: 0.7916018662519441 Кол-во деревьев: 10 Максим-ая глубина: 2
Accuracy на обуч-щей выборке: 0.8008298755186722 Кол-во деревьев: 10 Максим-ая глубина: 2
Accuracy на валид-ой выборке: 0.7931570762052877 Кол-во деревьев: 10 Максим-ая глубина: 3
Accuracy на обуч-щей выборке: 0.8008298755186722 Кол-во деревьев: 10 Максим-ая глубина: 3
Accuracy на валид-ой выборке: 0.7916018662519441 Кол-во деревьев: 10 Максим-ая глубина: 4
Accuracy на обуч-щей выборке: 0.8065352697095436 Кол-во деревьев: 10 Максим-ая глубина: 4
Accuracy на валид-ой выборке: 0.7993779160186625 Кол-во деревьев: 10 Максим-ая глубина: 5
Accuracy на обуч-щей выборке: 0.8205394190871369 Кол-во деревьев: 10 Максим-ая глубина: 5
Accuracy на валид-ой выборке: 0.7931570762052877 Кол-во деревьев: 10 Максим-ая глубина: 6
Accuracy н

### Вывод

- перебирая различные значения количества деревьев и максимальной глубины дерева, получили значение качества на валидационной выборке Accuracy: 0.8040435458786936, при максимальной глубине: 7 и количестве деревьев: 20.

- также проверили предсказания модели по  выборке, по которой модель обучалась. При количестве деревьев: 70 и максимальной глубине: 23 - Accuracy равно единице.

- как видим, на лицо переобучение модели, поскольку качество на валидационной выборке значительно ниже, чем на обучающей.

- при этом скорость работы модели медленная.

#### 5-bullet

### Часть 3. Исследование качества разных моделей:

3. Качество логистической регрессии.

- построим модель логистической регрессии, зафиксировав гиперпараметр random_state зафиксируем на одном значении.
- получим предсказания модели по валидационной выборке и получим качество модели.
- получим предсказания модели по обучающей выборке и получим качество модели, чтобы проверить на склонность к переобучению.

In [5]:
# сохраним модель логистической регрессии
model_lr = LogisticRegression(random_state=my_random_state, solver = 'lbfgs') 
model_lr.fit(features_train, target_train) # обучим модель 
# получим качество по валидационной выборке
result_1 = model_lr.score(features_valid, target_valid)  
# получим качество по обучающей выборке
result_2 = model_lr.score(features_train, target_train) 

#выведем на экран значение качества модели
print("Accuracy модели логистической регрессии на валидационной выборке:", result_1)
print("Accuracy модели логистической регрессии на обучающей выборке:", result_2)

Accuracy модели логистической регрессии на валидационной выборке: 0.7340590979782271
Accuracy модели логистической регрессии на обучающей выборке: 0.7546680497925311


### Вывод

- обучив модель на тестовой выборке, получили значение качества на валидационной выборке Accuracy: 0.7340590979782271.

- затем проверили предсказания модели по выборке, по которой модель обучалась. Accuracy модели логистической регрессии на обучающей выборке: 0.7546680497925311.

- у данной модели нет склонности к переобучению, получили близкие показатели Accuracy, но значения Accuracy невыскокие, что говорит скорее о недообучении модели.

- при этом скорость работы модели нормальная.

### Общий вывод по качеству моделей


- перебирая различные значения максимальной глубины дерева для модели решающее дерево, получили лучшее значение качества на валидационной выборке Accuracy: 0.7947122861586314 при максимальной глубине: 9.

- перебирая различные значения количества деревьев и максимальной глубины дерева для модели случайный лес, получили лучшее значение качества на валидационной выборке Accuracy: 0.8040435458786936, при максимальной глубине: 7 и количестве деревьев: 20.

- Наилучший показатель качества Accuracy на валидационной выборке показала модель случайный лес - 0.8, близкое к лучшему значению показала модель решающий лес - 0.79 и наихудший результат показала модель логистической регрессии - 0.75.

- При этом модели решающее дерево и случайный лес показали склонность к переобучению, поскольку дают точные предсказания по обучающей выборке и значительно хуже качества предсказания по валидационной выборке. Модель логистической регрессии напротив показала склонность к недообучению, поскольку показатели качества почти одинаково низкие, как на валидационной, так и на обучающей выборках.

- Нормальную скорость работы показали модели решающее дерево и логистическая регрессия и низкую скорость - случайный лес.

#### 6-bullet

### Часть 4. Проверка качества модели на тестовой выборке:

1. Проверка качества модели на тестовой выборке.

Ввиду того, что лучшее качество на валидационной выборке показала модель случайный лес, то проверим эту модель на тестовой выборке, предварительно обучив модель на обучающей с валидационной выборке.

In [6]:
# сохраним модель случайный лес и зададим лучшие гиперпараметры
model_rf = RandomForestClassifier(random_state=my_random_state, 
                                  n_estimators=20, max_depth=7)
model_rf.fit(features_train_valid, target_train_valid) # обучим модель
test_predictions = model_rf.predict(features_test) # получим предсказания 
train_valid_predictions = model_rf.predict(features_train_valid) # получим предсказания 

# выведем на экран качество 
print('Accuracy обучающей и валидационной выборки:', accuracy_score(target_train_valid, train_valid_predictions)) 
print('Accuracy тестовой выборки:', accuracy_score(target_test, test_predictions)) 

Accuracy обучающей и валидационной выборки: 0.8455853753403345
Accuracy тестовой выборки: 0.8133748055987559


### Вывод

- обучив модель случайный лес с лучшими гиперпараметрами, получили значение Accuracy на тестовой выборке: 0.8133748055987559 (Accuracy на обучающей с валидационной выборке: 0.8455853753403345).
- данное обстоятельство подтверждает, что "чем больше данных, тем лучше".
- полученный показатель Accuracy отвечает поставленной задаче добиться показателя Accuracy выше 0.75.